#Loading Wikipedia Dataset

In [1]:
!pip install -U transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 26.6 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.42.4
    Uninstalling transformers-4.42.4:
      Successfully uninstalled transformers-4.42.4


In [2]:
!pip install -U sentence-transformers

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [8]:
import json
from sentence_transformers import util
import gzip
import os
import torch

if not torch.cuda.is_available():
    print("Warning: No GPU found. Please add GPU to your notebook")

wikipedia_filepath = 'simplewiki-2020-11-01.jsonl.gz'

if not os.path.exists(wikipedia_filepath):
    util.http_get('http://sbert.net/datasets/simplewiki-2020-11-01.jsonl.gz', wikipedia_filepath)

  0%|          | 0.00/50.2M [00:00<?, ?B/s]

In [9]:
with gzip.open(wikipedia_filepath, 'rt', encoding='utf-8') as f:
  documents = []
  for line in f:
    data = json.loads(line) #Converting string lines in JSON file into dictionary in Python

    documents.append(data['paragraphs'][0])

print(len(documents))

169597


In [10]:
testing_words = ['india', 'north pole', 'nlp', 'natural language processing', 'linguistics',
                 'machine learning', 'artificial intelligence', 'cheetah', 'animal', 'jaguar']

In [11]:
new_documents = []
for doc in documents:
  for word in testing_words:
    if(word in doc.lower()):
      new_documents.append(doc)
      break
print(len(new_documents))

5872


In [12]:
new_documents[600]

'The Parazoa are an ancestral subkingdom of animals, literally translated as "beside the animals". Parazoans differ from their choanoflagellate ancestors in that they are macroscopic and have differentiated cells, but unlike "true animals" (Eumetazoa), they do not have tissues. Most people put Placozoa in Parazoa, but Placozoa can also be placed in the subkingdom Agnotozoa.'

#Inferencing Gemma Model (Quantized) from HF

In [3]:
from google.colab import userdata
hf_token = userdata.get('API_KEY')

In [4]:
from huggingface_hub import login

login(token=hf_token)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [5]:
!pip install bitsandbytes accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 8.0 MB/s eta 0:00:00


In [ ]:
!pip install -U bitsandbytes

In [6]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(load_in_8bit=True)

tokenizer = AutoTokenizer.from_pretrained("google/gemma-2-9b-it")
llm_model = AutoModelForCausalLM.from_pretrained(
    "google/gemma-2-9b-it",
    quantization_config=quantization_config,
)

input_text = "Write me a poem about Machine Learning."
input_ids = tokenizer(input_text, return_tensors="pt").to("cuda")

outputs = llm_model.generate(**input_ids, max_new_tokens=32)
print(tokenizer.decode(outputs[0]))

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/47.0k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

...

model-00003-of-00004.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/3.67G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

<bos>Write me a poem about Machine Learning.

In silicon valleys, where data flows,
A new intelligence, silently grows.
Machine Learning, a name whispered low,
Algorithms dance, where patterns


In [7]:
outputs[0]

tensor([     2,   5559,    682,    476,  19592,   1105,  13403,  14715, 235265,
           109,    886,  43810,  58087, 235269,   1570,   1423,  22087, 235269,
           108, 235280,    888,  17273, 235269,  61920,  26075, 235265,    108,
         24911,  14715, 235269,    476,   1503,  45959,   3178, 235269,    108,
        139972,  11877, 235269,   1570,  12136], device='cuda:0')

#LLM Embedder using General Text Embedding Model (GTE-BASE)

In [13]:
!pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 997.8/997.8 kB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 384.8/384.8 kB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.4/140.4 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 11.8 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0


In [14]:
!pip install -qU "langchain-chroma>=0.1.2"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 584.3/584.3 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 35.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 39.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.6/67.6 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 52.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.5/61.5 kB 3.3 MB/s eta 0:00:00
  

In [15]:
!pip install langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 40.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 3.8 MB/s eta 0:00:00


In [16]:
!pip install -U langchain-huggingface

In [17]:
from langchain_huggingface import HuggingFaceEmbeddings

embed_model = HuggingFaceEmbeddings(model_name='thenlper/gte-base')

modules.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/68.1k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/618 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/219M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/314 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [18]:
print(embed_model)

client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
) model_name='thenlper/gte-base' cache_folder=None model_kwargs={} encode_kwargs={} multi_process=False show_progress=False


#Document Splitting/Chunking and Storing in Vector DB

In [19]:
from langchain_chroma import Chroma

from langchain.text_splitter import RecursiveCharacterTextSplitter

In [20]:
from langchain.docstore.document import Document

docs = [Document(page_content=doc) for doc in new_documents]

In [21]:
docs[:5]

[Document(page_content="The integumentary system is everything covering the outside of an animal's body. This account is written mostly with people in mind, but it applies more widely."),
 Document(page_content='The electric eel, "Electrophorus electricus", is a species of fish. It can weigh up to six pounds. It can give electric shocks of up to six hundred fifty watts of electricity. The animal uses these shocks both for hunting and to defend itself. It lives in the stagnant muddy river bottoms of the Orinoco and the Amazon, and uses low-voltage electric fields to find its prey.'),
 Document(page_content='The term Far East means countries of East Asia. It became used in the English language during the period of the British Empire for lands to the east of British India.'),
 Document(page_content="A chakra is an energy center in the human body as taught in Indian yoga and some related cultures. Chakras are also believed in by some New Age adherents. Chakra means ‘Aneel or wheel' in Sans

In [22]:
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=300)
chunked_docs = splitter.split_documents(docs)

In [23]:
chunked_docs[:5]

[Document(page_content="The integumentary system is everything covering the outside of an animal's body. This account is written mostly with people in mind, but it applies more widely."),
 Document(page_content='The electric eel, "Electrophorus electricus", is a species of fish. It can weigh up to six pounds. It can give electric shocks of up to six hundred fifty watts of electricity. The animal uses these shocks both for hunting and to defend itself. It lives in the stagnant muddy river bottoms of the Orinoco and the Amazon, and uses low-voltage electric fields to find its prey.'),
 Document(page_content='The term Far East means countries of East Asia. It became used in the English language during the period of the British Empire for lands to the east of British India.'),
 Document(page_content="A chakra is an energy center in the human body as taught in Indian yoga and some related cultures. Chakras are also believed in by some New Age adherents. Chakra means ‘Aneel or wheel' in Sans

#Initializing Chroma DB

In [24]:
from langchain_chroma import Chroma

vector_store = Chroma(
    collection_name="simple_wiki_db",
    embedding_function=embed_model,
    persist_directory="./simple_wiki_db",
)

In [25]:
vector_store.add_documents(documents=chunked_docs)

['d64c5242-1033-4a58-bb47-6899bcb4b3c5',
 '5bb644bc-216b-4352-a1be-9e7a5d861ef0',
 'c1339f30-8a6d-4015-8101-55a984092fd4',
 'bb5a28db-b067-4d76-b966-3fa2adbaf473',
 '01e4ffd6-aed8-40a5-a084-ce61617032ab',
 'aa5a9a43-dd3c-4ef0-bcff-9c75ec5dc280',
 '7f516920-b4d0-4599-aeb1-1ab1c2a53202',
 'e48e7c82-311a-4c0e-b73c-ea4cb976ad57',
 'ea7e0769-1239-4e00-a9e8-12fdf9455a23',
 'cc87d6a7-7fcf-4d27-b298-33f52c560ee0',
 'b8794107-e3a4-4591-aa31-6fc5afe2a807',
 '6f9207fb-6d85-4605-b0a2-5f0625a2cced',
 'a3796156-c535-4eb8-a652-3dd19091df5e',
 '15efa64c-60f9-44f3-82e8-5404293528bc',
 'e2f04785-d54a-4a1d-bc8f-1ad836baa897',
 '95af4e98-7eb6-4ab8-95f6-a371cba32b0e',
 'f5ef53d8-3d8c-4689-8da9-5649c646f803',
 'e39b9637-9a1e-4766-bd2a-a3e1f2c81091',
 'aa4a70ef-b8d3-4c25-873d-33f95fb8a63c',
 '39131ee9-51ae-49f4-b6ba-feeeef036a45',
 '4f6fe862-8bed-42bf-aa97-73f553da6826',
 '4948525d-f026-4847-a59c-968199ea7c40',
 '8f071080-dcb3-4fbd-959f-241277dd0081',
 'b66ec7ff-f425-4270-9d12-3137d1498f9f',
 'f3962767-f795-

In [26]:
similarity_retriever = vector_store.as_retriever(search_type="similarity_score_threshold",
search_kwargs={"k": 5, "score_threshold": 0.2})

In [27]:
similarity_retriever.invoke('what is the capital of India?')

[Document(page_content="Mumbai (previously known as Bombay until 1996) is a natural harbor on the west coast of India, and is the capital city of Maharashtra state. It is India's largest city, and one of the world's most populous cities. It is the financial capital of India. The city is the second most-populous in the world. It has approximately 13 million people. Along with the neighboring cities of Navi Mumbai and Thane, it forms the world's 4th largest urban agglomeration. They have around 19.1 million people."),
 Document(page_content="Kolkata (spelled Calcutta before 1 January 2001) is the capital city of the Indian state of West Bengal. It is the second largest city in India after Mumbai. It is on the east bank of the River Hooghly. When it is called Calcutta, it includes the suburbs. This makes it the third largest city of India. This also makes it the world's 8th largest metropolitan area as defined by the United Nations. Kolkata served as the capital of India during the Britis

#Building RAG Pipeline

In [28]:
!pip install langchainhub

In [ ]:
# from langchain import hub, PromptTemplate

# prompt_template = hub.pull("rlm/rag-prompt")
# prompt_template

ChatPromptTemplate(input_variables=['context', 'question'], metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"))])

In [29]:
from langchain_core.prompts import ChatPromptTemplate

prompt = """You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that you don't know.
Keep the answer upto 5 lines unless the user asks for more information

Question:
{question}

Context:
{context}

Answer:
"""

prompt_template = ChatPromptTemplate.from_template(prompt)

In [30]:
prompt_template.messages[0].prompt

PromptTemplate(input_variables=['context', 'question'], template="You are an assistant for question-answering tasks.\nUse the following pieces of retrieved context to answer the question.\nIf you don't know the answer, just say that you don't know.\nKeep the answer upto 5 lines unless the user asks for more information\n\nQuestion:\n{question}\n\nContext:\n{context}\n\nAnswer:\n")

In [85]:
from langchain_huggingface import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from transformers import pipeline
from langchain_core.output_parsers import StrOutputParser

text_generation_pipeline = pipeline(
    model=llm_model,
    tokenizer=tokenizer,
    task="text-generation",
    return_full_text=False,
    max_new_tokens=50,
)

llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

In [54]:
from langchain_core.runnables import RunnablePassthrough

def format_docs(docs):
  return "\n\n".join(doc.page_content for doc in docs)

qa_rag_chain = (
{
"context": (similarity_retriever
                |
            format_docs),
"question": RunnablePassthrough()
}
      |
  prompt_template
      |
    llm
      |
    StrOutputParser()
)

In [38]:
query = "What is the capital of India?"
result = qa_rag_chain.invoke(query)
print(result)

Human: You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that you don't know.
Keep the answer upto 5 lines unless the user asks for more information

Question:
What is the capital of India?

Context:
Mumbai (previously known as Bombay until 1996) is a natural harbor on the west coast of India, and is the capital city of Maharashtra state. It is India's largest city, and one of the world's most populous cities. It is the financial capital of India. The city is the second most-populous in the world. It has approximately 13 million people. Along with the neighboring cities of Navi Mumbai and Thane, it forms the world's 4th largest urban agglomeration. They have around 19.1 million people.

Kolkata (spelled Calcutta before 1 January 2001) is the capital city of the Indian state of West Bengal. It is the second largest city in India after Mumbai. It is on the east bank of the River 

In [39]:
query = "What is Machine Learning?"
result = qa_rag_chain.invoke(query)
print(result)

Human: You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that you don't know.
Keep the answer upto 5 lines unless the user asks for more information

Question:
What is Machine Learning?

Context:
Machine learning gives computers the ability to learn without being explicitly programmed (Arthur Samuel, 1959). It is a subfield of computer science.

In machine learning, supervised learning is the task of inferring a function from labelled training data. The results of the training are known beforehand, the system simply learns how to get to these results correctly. Usually, such systems work with vectors. They get the training data and the result of the training as two vectors and produce a "classifier". Usually, the system uses inductive reasoning to generalize the training data.

Deep learning (also called deep structured learning or hierarchical learning) is a kind of machine le

In [40]:
#Can't retain memory of past interactions
query = "Tell me more about it in detail"
result = qa_rag_chain.invoke(query)
print(result)

Human: You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that you don't know.
Keep the answer upto 5 lines unless the user asks for more information

Question:
Tell me more about it in detail

Context:
Tipu's Tiger (a.k.a. Tippoo's Tiger) is an 18th century moving model, musical instrument and an art object. It was made for Tipu, the ruler of Mysore in India. It shows a tiger killing a European soldier. It is in the Victoria and Albert Museum in London.

The integumentary system is everything covering the outside of an animal's body. This account is written mostly with people in mind, but it applies more widely.

Display is a form of animal behaviour, linked to survival of the species in various ways.

Tripe is part of an animal's stomach often served as food.

Timila, thimila or paani, (Malayalam: തിമില) is an hour-glass shaped drum used in Kerala, South India. Usually, the in

#Conversational RAG

In [86]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

#Reformulating the latest user query based upon past interactions for vector db to retrieve relevant contexts based upon it

rephrase_system_prompt = """Given a chat history and the latest user question
which might reference context in the chat history, formulate a standalone question
which can be understood without the chat history. Do NOT answer the question,
just reformulate it if needed and otherwise return it as is."""

rephrase_prompt = ChatPromptTemplate.from_messages(
[
("system", rephrase_system_prompt),
MessagesPlaceholder("chat_history"),
("human", "{input}"),
]
)

history_aware_retriever = create_history_aware_retriever(
llm, similarity_retriever, rephrase_prompt
)

history_aware_retriever

RunnableBinding(bound=RunnableBranch(branches=[(RunnableLambda(lambda x: not x.get('chat_history', False)), RunnableLambda(lambda x: x['input'])
| VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x7d46138d0880>, search_type='similarity_score_threshold', search_kwargs={'k': 5, 'score_threshold': 0.2}))], default=ChatPromptTemplate(input_variables=['chat_history', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='Given a chat history and the latest user question\nwhich might reference context in the chat history, formulate a standalone question\nwhich 

In [87]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

qa_system_prompt = """You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that you don't know.
Keep the answer upto 5 lines unless the user asks for more information

Question: {input}

Context:
{context}
"""

qa_prompt = ChatPromptTemplate.from_messages(
[
("system", qa_system_prompt),
MessagesPlaceholder("chat_history"),
("human", "{input}"),
]
)

question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

qa_rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)
qa_rag_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableBranch(branches=[(RunnableLambda(lambda x: not x.get('chat_history', False)), RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x7d46138d0880>, search_type='similarity_score_threshold', search_kwargs={'k': 5, 'score_threshold': 0.2}))], default=ChatPromptTemplate(input_variables=['chat_history', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='Given a chat history and the latest user question\nwhich might reference 

In [88]:
chat_history = []

question = "What is Machine Learning?"
response = qa_rag_chain.invoke({"input": question, "chat_history": chat_history})
print(response['answer'])



Assistant: Machine learning gives computers the ability to learn without being explicitly programmed. It is a subfield of computer science. 





In [77]:
for chunk in qa_rag_chain.stream({"input": question, "chat_history": chat_history}):
  print(chunk)

{'input': 'What is Machine Learning?', 'chat_history': []}
{'context': [Document(page_content='Machine learning gives computers the ability to learn without being explicitly programmed (Arthur Samuel, 1959). It is a subfield of computer science.'), Document(page_content='In machine learning, supervised learning is the task of inferring a function from labelled training data. The results of the training are known beforehand, the system simply learns how to get to these results correctly. Usually, such systems work with vectors. They get the training data and the result of the training as two vectors and produce a "classifier". Usually, the system uses inductive reasoning to generalize the training data.'), Document(page_content='Deep learning (also called deep structured learning or hierarchical learning) is a kind of machine learning, which is mostly used with certain kinds of neural networks. As with other kinds of machine-learning, learning sessions can be unsupervised, semi-supervis

In [89]:
from langchain_core.messages import HumanMessage, AIMessage

chat_history.extend([HumanMessage(content=question),
AIMessage(content=response["answer"])])
chat_history

[HumanMessage(content='What is Machine Learning?'),
 AIMessage(content='\n\nAssistant: Machine learning gives computers the ability to learn without being explicitly programmed. It is a subfield of computer science. \n\n\n')]

In [90]:
question = "Tell me more about it in detail"
response = qa_rag_chain.invoke({"input": question, "chat_history": chat_history})
print(response['answer'])



Assistant: Machine learning involves training computers on data to identify patterns and make predictions. 

There are different types of machine learning:

* **Supervised learning:**  The algorithm is trained on labeled data, meaning each data point has a known


In [91]:
chat_history.extend([HumanMessage(content=question),
AIMessage(content=response["answer"])])
chat_history

[HumanMessage(content='What is Machine Learning?'),
 AIMessage(content='\n\nAssistant: Machine learning gives computers the ability to learn without being explicitly programmed. It is a subfield of computer science. \n\n\n'),
 HumanMessage(content='Tell me more about it in detail'),
 AIMessage(content='\n\nAssistant: Machine learning involves training computers on data to identify patterns and make predictions. \n\nThere are different types of machine learning:\n\n* **Supervised learning:**  The algorithm is trained on labeled data, meaning each data point has a known')]

In [92]:
response

{'input': 'Tell me more about it in detail',
 'chat_history': [HumanMessage(content='What is Machine Learning?'),
  AIMessage(content='\n\nAssistant: Machine learning gives computers the ability to learn without being explicitly programmed. It is a subfield of computer science. \n\n\n'),
  HumanMessage(content='Tell me more about it in detail'),
  AIMessage(content='\n\nAssistant: Machine learning involves training computers on data to identify patterns and make predictions. \n\nThere are different types of machine learning:\n\n* **Supervised learning:**  The algorithm is trained on labeled data, meaning each data point has a known')],
 'context': [Document(page_content='Machine learning gives computers the ability to learn without being explicitly programmed (Arthur Samuel, 1959). It is a subfield of computer science.'),
  Document(page_content='Artificial intelligence (AI) is the ability of a computer program or a machine to think and learn. It is also a field of study which tries to

#Returning Sources

In [93]:
chat_history = []

In [94]:
question = "which is the fastest animal?"
response = qa_rag_chain.invoke({"input": question, "chat_history": chat_history})
print(response['answer'])
print('Sources:')
for document in response['context']:
  print(document)
  print()



Assistant: The cheetah is the fastest land animal and can run up to 112 kilometers per hour. 



Sources:
page_content='A cheetah ("Acinonyx jubatus") is a medium large cat which lives in Africa. It is the fastest land animal and can run up to 112 kilometers per hour for a short time. Most cheetahs live in the savannas of Africa. There are a few in Asia. Cheetahs are active during the day, and hunt in the early morning or late evening.'

page_content='The fin whale ("Balaenoptera physalus") is a huge baleen whale. It is the second largest animal on Earth (the blue whale is the biggest). This whale is sometimes called the "greyhound of the sea" because of its fast swimming speed; it can swim up to 23 mph (37 km/hr) in short bursts. The fin whale is also called the finback, finner, razorback, common rorqual, and herring whale.'

page_content='A Greyhound is a dog breed that is skinny, has long legs, and runs faster than any other breed of dog. It is one of the oldest breeds of dogs. Fo

In [95]:
chat_history.extend([HumanMessage(content=question),
AIMessage(content=response["answer"])])
chat_history

[HumanMessage(content='which is the fastest animal?'),
 AIMessage(content='\n\nAssistant: The cheetah is the fastest land animal and can run up to 112 kilometers per hour. \n\n\n')]

In [96]:
question = "Tell me more, including different types of this animal and their details"
response = qa_rag_chain.invoke({"input": question, "chat_history": chat_history})
print(response['answer'])
print('Sources:')
for document in response['context']:
  print(document)
  print()



Assistant: 
There are several types of cheetahs. The South African Cheetah is the most abundant, with over 6,000 individuals in the wild. The Asiatic cheetah is critically endangered. 

Sources:
page_content='A cheetah ("Acinonyx jubatus") is a medium large cat which lives in Africa. It is the fastest land animal and can run up to 112 kilometers per hour for a short time. Most cheetahs live in the savannas of Africa. There are a few in Asia. Cheetahs are active during the day, and hunt in the early morning or late evening.'

page_content='The South African Cheetah ("Acinonyx jubatus jubatus"), also known as Namibian Cheetah, is the nominate subspecies of cheetah native to Southern Africa. It is the most abundant subspecies estimated at more than 6,000 individuals in the wild. Since 1990 and onwards, the population was estimated at approximately 2,500 individuals in Namibia, until 2015, the cheetah population has been increased to more than 3,500 in the country.'

page_content='The As